In [ ]:
from tkinter import *
import tkinter as tk
from tkinter import ttk
import pandas as pd
import os 
import cv2
import keras as ks
import tensorflow as tf
import numpy as np
from tkinter import filedialog 
from PIL import ImageTk, Image
from keras.applications.vgg16 import VGG16
from __future__ import print_function, division
from builtins import range, input
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
from glob import glob
from keras.utils import to_categorical
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
    
def openfilename():
    global filename1
    filename1 = filedialog.askopenfilename(title="Choose an image",
                                       filetypes=(("all files", "*.*"), ("png files", "*.png")))
    global img20
    img20 = ImageTk.PhotoImage(Image.open(filename1))
    panel = Label(f3, image=img20)
    panel.grid(row=1,column=4)    
        
def  Areas():
    labels = ['Forehead','Chest','upper back','Shoulders','face','Underarms','Upper arms(shoulder to elbow)','Lower Legs or leg','Adult','Children','Elbows','Knees','Cheeks','Back of knees','Hands','Feet','Lower arms','Upper leg or thighs','Buttocks','Around eyes','Stomach','Neck','Folds of skin','Around Nose','Around mouth','Groin area','Chin','Around ears','Beard','Nails','Genital areas']
    global l1
    l1=[]
    for i in range(0,27):
        l1.append(IntVar())
        Checkbutton(f1, text=labels[i], variable=l1[i]).grid(row=i,column=0, sticky=W)
    for i in range(27,31):
        l1.append(IntVar())
        Checkbutton(f1, text=labels[i], variable=l1[i]).grid(row=i-27,column=1, sticky=W)
    Button(f1, text="Next",font = ("Arial Bold",10), command = Symptoms).grid(row=28,column=3)
    f1.tkraise()
    
def  Symptoms():
    labels = ['Itching','Burns','swelling of mouth','Triggered with light','Triggered with heat','Ulcer','Redness of skin','Coloured sweat','Rash','Joint pain','Hair loss','Fever','Mouth sores','fatigue','eye problems','Cough','Headache','Darker skin','Enlarged skin']
    global l2
    l2=[]
    for i in range(0,19):
        l2.append(IntVar())
        Checkbutton(f2, text=labels[i], variable=l2[i]).grid(row=i,column=0, sticky=W)
    Button(f2, text="Detect",font = ("Arial Bold",10), command = detect).grid(row=28,column=3)
    f2.tkraise()
    

def classify():

    f4.tkraise()
    final = Label(f4, text = str(result)).grid( row = 4, column = 1)
    

def detect():
    arr1=[]
    arr2=[]
    for i in range(0,31):
        arr1.append(l1[i].get())
    for i in range(0,19):
        arr2.append(l2[i].get())
    df1=pd.read_csv('Disease Occurance location.csv')
    df2=pd.read_csv('Symptoms.csv')
    
    #Take all columns where arr1 is 1. All the disease which lies in that column extract that. Do that for all columns and
    #take intersection of that. Do same thing for both symptoms and occurance and take intersection of both of them.
    possible_diseases=[]
    diseases1 = df1.iloc[:,0].tolist()
    for i in range(0,31):
        if arr1[i]==1:
            df = df1[df1[df1.columns[i+1]]==1].iloc[:,0].tolist()
            for j in range(len(df)):
                possible_diseases.append(df[j])
            
            diseases1 = list(set(possible_diseases) & set(diseases1))
            possible_diseases=[]
            
            
            
    possible_diseases=[]
    diseases2 = df2.iloc[:,0].tolist()
    for i in range(0,19):
        if arr2[i]==1:
            df = df2[df2[df2.columns[i+1]]==1].iloc[:,0].tolist()
            for j in range(len(df)):
                possible_diseases.append(df[j])
            
            diseases2 = list(set(possible_diseases) & set(diseases2))
            possible_diseases=[]
    
    global diseases
    diseases = list(set(diseases1) & set(diseases2))
    
    #creating training data
    dir = r"C:\Users\91852\Desktop\Projects\ML_projects\Skin_disease_detection\photos"
    img_size = 400
    Training_data = []
    def create_Training_data():
        for category in diseases:
            path = os.path.join(dir,category)
            class_num = diseases.index(category)
            for img in os.listdir(path):
                try:
                    img_array = cv2.imread(os.path.join(path,img))
                    new_array = cv2.resize(img_array,(img_size,img_size))
                    Training_data.append([new_array,class_num])
                except Exception as e:
                      pass
    create_Training_data()
    y = [i[1] for i in Training_data] 
    x = [i[0] for i in Training_data] 
    x = np.array(x)
    y = np.array(y)

    vgg = VGG16(input_shape=(400,400,3), weights='imagenet', include_top=False)


    for layer in vgg.layers:
        layer.trainable = False

    layer1 = Flatten()(vgg.output)
    layer2 = Dense(256,activation = 'relu')(layer1)
    prediction = Dense(len(diseases), activation='softmax')(layer2)
    


    # create a model object
    model = Model(inputs=vgg.input, outputs=prediction)
    model.compile(
      loss='categorical_crossentropy',
      optimizer='adam',
      metrics=['accuracy']
    )
    y = to_categorical(y,len(diseases))
    model.fit(x, y, epochs = 3)
    
    #After getting list of items
    
    f3.tkraise()
    Intro = Label(f3, text="Skin disease - Detection and cure",font = ("Arial Bold",20))
    Intro.grid(row=0,column=0)
    Button(f3, text ='select image', command = openfilename).grid( row = 1, column = 1)
    Button(f3, text ='Classify', command = classify).grid( row = 1, column = 2)
    img_size = 400
    img_array = cv2.imread(filename1)
    new_array = cv2.resize(img_array,(img_size,img_size))
    x=tf.keras.utils.normalize(new_array)
    x = np.array(x).reshape(-1, img_size,img_size, 3)
    
    global result
    result = diseases[np.argmax(model.predict(x))]
    
    Button(f3, text ='Classify', command = classify).grid( row = 1, column = 2) 


    


    
root = tk.Tk()
global f1
global f2
global f3
global f4
f1 = Frame(root)
f2 = Frame(root)
f3 = Frame(root)
f4 = Frame(root)
for frame in (f1,f2,f3,f4):
    frame.grid(row = 0, column = 0, sticky='nsew')
Areas()
root.mainloop()

